In [1]:
import pathlib

In [2]:
import pandas as pd

In [3]:
train_days = 1913
test_days = 28

In [4]:
root_dir_path = pathlib.Path("..")
data_dir_path = root_dir_path / "data"
raw_dir_path = data_dir_path / "raw"
sample_submission_path = raw_dir_path / "sample_submission.csv"
models_dir_path = root_dir_path / "models"
prediction_path = models_dir_path / "prediction.parquet"
submission_path = models_dir_path / "submission.csv.gz"

In [5]:
sample_submission = pd.read_csv(sample_submission_path)
prediction = pd.read_parquet(prediction_path)

In [6]:
is_evaluation = prediction["id"].str.endswith("_evaluation")
intermediate = prediction["d"].str[2:]
intermediate = intermediate.astype("int32")
intermediate -= train_days
intermediate[is_evaluation] -= test_days
prediction["d"] = intermediate

In [7]:
submission = pd.pivot(prediction, index = 'id', columns = 'd', values = 'demand')

In [8]:
submission.reset_index(inplace=True)

In [9]:
submission.columns = ["id"] + [f"F{i + 1}" for i in range(test_days)]

In [10]:
submission = pd.merge(sample_submission["id"], submission, how="left")

In [11]:
submission.shape

(60980, 29)

In [12]:
submission.to_csv(submission_path, index=False)